In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
import numpy as np
from cmdstanpy import CmdStanModel
from baynes import FitPlotter, get_compiler_kwargs, get_stan_file, standard_analysis, hdi
import pandas as pd

sns.set_style('ticks')
sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=1.6)
plt.rc("axes.spines", top=False, right=False)

In [ ]:
N = 21
x = np.linspace(0, 10, N)

alpha = 0.5
beta = 0.4
sigma = 0.5

y = alpha + beta * (x + np.random.normal(0, sigma, N))
plt.plot(x, y)

data = {'N': N, 'x': x, 'y': y, 'prior': 0}
fplot = FitPlotter(output_dir="/home/pietro/work/TESI/thesis/figures/ch2/linear/", output_format='.pdf', fig_ratio=1.2, fig_scale=6)

In [ ]:
model = CmdStanModel(stan_file=get_stan_file('linear_simple.stan'),
                     **get_compiler_kwargs())
                     
model_noalpha = CmdStanModel(stan_file=get_stan_file('linear_no_intercept.stan'),
                            **get_compiler_kwargs())

In [ ]:
fit = model.sample(data)
fplot.add_fit(fit, 'with intercept')
fit_noalpha = model_noalpha.sample(data)
fplot.add_fit(fit_noalpha, 'no intercept')


In [ ]:
ax = fplot.new_figure('ppc_compare').subplots()
df_tot = pd.DataFrame()
for title, f in fplot.fits.items():
    df = f.draws_pd('y_rep').transpose()
    df['x']=x
    df['fit'] = title
    df_tot = pd.concat([df, df_tot])

#sns.set_palette(sns.cubehelix_palette(hue=1))
ax.plot(x, y, label='data', color='black')
ax = sns.lineplot(df_tot.melt(id_vars=['x', 'fit']), x='x', y='value', hue='fit', errorbar=hdi, ax=ax, err_kws={'alpha':1}, alpha=0)
plt.legend(bbox_to_anchor=(1.2, 0.4), facecolor='white', edgecolor='white')
ax.set_ylabel('y')
fplot.save_figures()

In [ ]:
df = az.compare({'no intercept': fit_noalpha, 'with intercept':fit}, ic='waic')
print(df[['elpd_waic', 'se', 'p_waic']].to_latex(float_format="%.2f"))

In [ ]:
ax = fplot.new_figure('loo_khat').subplots()
loo = az.loo(fit_noalpha)
ax = az.plot_khat(loo, label = 'no intercept', ax=ax, markersize=100)
loo = az.loo(fit)
az.plot_khat(loo, label = 'with intercept', ax=ax, markersize=100, color='orange', marker='o')
plt.legend(bbox_to_anchor=(1.05, 0.5), facecolor='white', edgecolor='white')
fplot.save_figures()

In [ ]:
df = az.compare({'no intercept': fit_noalpha, 'with intercept':fit})
print(df[['elpd_loo', 'se', 'p_loo']].to_latex(float_format="%.2f"))